In [1]:
!pip install zeugma

     |████████████████████████████████| 462 kB 5.4 MB/s 
  Created wheel for zeugma: filename=zeugma-0.49-py3-none-any.whl size=8823 sha256=7f2d4b2b504243599eb5f234b4dc1844457e82115c7dc0274f8bb4c7756fce56
  Stored in directory: /root/.cache/pip/wheels/1d/47/5b/2a59a79706cc9340c72fd6a7bfc20e7ebcab849c88c38fdfa0
Successfully built zeugma


# Exploring and Cleaning the data

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('train_aug_in_kag_org.csv')

In [3]:
data.head(10)

,title,text,title_text,label
0,squirrel financial wellbeing,A Squirrel account makes it easy to stick to your budget and save. Start for free today!,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,senior insurance marketing,"Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.","senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,3i financial group inc.,"A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more","3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,brownstone insurance,"Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.","brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending,Learn more about Pacificwide Lending and our commercial & residential loan services in California.,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
5,city auto finance,"City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.","city auto finance. City Auto Finance is the Southeast's most trusted used-car floor plan financing company with locations in Memphis, TN; Mobile, AL; Huntsville, AL and Murfreesboro, TN. We provide you with competitive rates and the capability to make payments online.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
6,"veterans financial, inc.","VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost about benefits","veterans financial, inc.. VA Aid and Attendance benefit can help pay for assisted living, home care or nursing home. Veterans Financial Inc can advise you at no cost 

In [4]:
data.shape

(34953, 4)

In [5]:
data.isnull().sum()

title         42
text           0
title_text     0
label          0
dtype: int64

In [6]:
data = data.iloc[:, 2:]

In [7]:
data.head()

,title_text,label
0,squirrel financial wellbeing. A Squirrel account makes it easy to stick to your budget and save. Start for free today!,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
1,"senior insurance marketing. Welcome to the home page of Senior Insurance Marketing, a full service brokerage specializing in Medicare and senior insurance products. Located in Lincoln, NE.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
2,"3i financial group inc.. A Small Boutique Firm Dedicated to Achieving Our Clients Goals.#sep#inform.invest.insure Get a Free, No Obligation Quote forAllYour Insurance Needs. 3i Financial offers a wide variety of insurance products to help suit your needs. Choose from individual, family, and group. GET A QUOTE TODAY Travel Insurance Get peace of mind with comprehensive travel insurance. Get a free quote today! Health & Dental Get affordable health Read more",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
3,"brownstone insurance. Brownstone Insurance provides master insurance policies for multi-unit buildings throughout Massachusetts, as well as personal insurance products such as renter's insurance and homeowner's insurance.#sep#Brownstone Insurance provides master insurance policies for multi-unit \nbuildings throughout Massachusetts, as well as personal insurance products \nsuch as renter's insurance and homeowner's insurance.",payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing
4,pacificwide lending. Learn more about Pacificwide Lending and our commercial & residential loan services in California.,payment-credit-card-balance-euro-dollars-credit-debit-receipt-billing


# Preparing our data for modeling 

In [8]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# text
text_corpus = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus.append(row)

In [10]:
len(text_corpus)

34953

In [11]:
text_corpus[0]

'squirrel financial wellbeing squirrel account make easy stick budget save start free today'

_Our Bag of Words_

In [12]:
from zeugma.embeddings import EmbeddingTransformer

In [14]:
text_em = EmbeddingTransformer('glove')   
X_text = text_em.transform(text_corpus)

[==================================================] 100.0% 104.8/104.8MB downloaded


In [43]:
text_em.transform(['I am perfect']).shape

(1, 25)

In [15]:
X_text.shape

(34953, 25)

In [16]:
X = X_text.copy()
y = data.iloc[:, 1].values

In [17]:
X.shape, y.shape

((34953, 25), (34953,))

In [18]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [19]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

# Splitting the data

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [21]:
X_train.shape, X_val.shape

((27962, 25), (6991, 25))

# Modeling

In [22]:
from time import time 

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

In [23]:
models = [
    ('GradientBoostingClassifier',GradientBoostingClassifier()),
    ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('LightGBM', LGBMClassifier())
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG', 'Time_taken'])
for model_name, model in models:
    start = time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    end = time()
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_val, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_val, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_val, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_val, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_val, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_val, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_val, y_pred, average='weighted')*100
    record['Time_taken'] = end-start
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)

In [24]:
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG,Time_taken
2,LightGBM,69.389215,64.430635,69.398576,51.702544,69.389215,55.160144,68.699564,18.954480
0,GradientBoostingClassifier,66.113575,50.968403,65.401409,45.167635,66.113575,46.809542,65.218633,495.467178
1,XGBoost,65.312545,53.911012,64.942374,44.596055,65.312545,46.027701,63.978130,73.966973


In [25]:
# saving the models
import joblib

joblib.dump(models[0][1], 'GradientBoostingClassifier.sav')
joblib.dump(models[1][1], 'XGBoost.sav')
joblib.dump(models[2][1], 'LightGBM.sav')

['LightGBM.sav']

# Testing on test_data

In [26]:
test_data = pd.read_csv('test_in_kag_org.csv')

In [27]:
test_data.shape

(6501, 4)

In [28]:
test_data.isnull().sum()

title         0
text          0
title_text    0
label         0
dtype: int64

In [29]:
test_data = test_data.iloc[:, 2:]

# Preparing our data for modeling 

In [30]:
# text
text_corpus_test = []
lm = WordNetLemmatizer()
words = stopwords.words('english')

for i in range(test_data.shape[0]):
    row = re.sub('[^a-zA-Z\s]', '', test_data.iloc[i, 0]).lower()
    row = row.split()  
    row = [lm.lemmatize(word) for word in row if word not in words]
    row = ' '.join(row)
    text_corpus_test.append(row)

In [31]:
len(text_corpus_test)

6501

In [32]:
text_corpus_test[0]

'centralized hub procore offer ability communicate team get full view project one place assign task stay connected google map google doc'

_Our Bag of Words_

In [33]:
X_text_test = text_em.transform(text_corpus_test)

In [34]:
X_text_test.shape

(6501, 25)

In [35]:
X_test = X_text_test.copy()
y_test = test_data.iloc[:, 1].values

In [36]:
X_test.shape, y_test.shape

((6501, 25), (6501,))

In [37]:
y_test = le.transform(y_test)

In [38]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [39]:
import joblib

GB_model = joblib.load('GradientBoostingClassifier.sav')
XGB_model = joblib.load('XGBoost.sav') 
LGBM_model = joblib.load('LightGBM.sav')

In [40]:
models = [
    ('GradientBoostingClassifier',GB_model),
    ('XGBoost', XGB_model),
    ('LightGBM', LGBM_model)
]

results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision-MacroAVG', 'Precision-WeightedAVG', 
                                  'Recall-MacroAVG', 'Recall-WeightedAVG', 'F1_score-MacroAVG','F1_score-WeightedAVG'])
for model_name, model in models:
    y_pred = model.predict(X_test)
    
    record = {}
    record['Model'] = model_name
    record['Accuracy'] = accuracy_score(y_test, y_pred)*100
    record['Precision-MacroAVG'] = precision_score(y_test, y_pred, average='macro')*100
    record['Precision-WeightedAVG'] = precision_score(y_test, y_pred, average='weighted')*100
    record['Recall-MacroAVG'] = recall_score(y_test, y_pred, average='macro')*100
    record['Recall-WeightedAVG'] = recall_score(y_test, y_pred, average='weighted')*100
    record['F1_score-MacroAVG'] = f1_score(y_test, y_pred, average='macro')*100
    record['F1_score-WeightedAVG'] = f1_score(y_test, y_pred, average='weighted')*100
    
    results = results.append(record, ignore_index=True)
    
results = results.sort_values(by='Accuracy', ascending=False)
results

,Model,Accuracy,Precision-MacroAVG,Precision-WeightedAVG,Recall-MacroAVG,Recall-WeightedAVG,F1_score-MacroAVG,F1_score-WeightedAVG
2,LightGBM,74.003999,46.278859,74.651647,47.372659,74.003999,45.832003,74.112956
0,GradientBoostingClassifier,71.850485,40.345016,73.123019,45.071234,71.850485,41.968650,72.183828
1,XGBoost,71.773573,44.587822,72.849661,45.617519,71.773573,43.524013,71.908974
